# W04B — Índices y ordenamiento: cuándo ayudan y cuándo NO (DuckDB)

**Objetivo:** aprender 3 ideas con evidencia (planes):
1) **Zonemaps (min-max)**: ya existen “gratis”; el **orden** de los datos importa.
2) **ART index** (`CREATE INDEX`): ayuda en consultas **muy selectivas** (tipo “needle in haystack”).
3) Índices **NO arreglan** consultas de **JOIN + GROUP BY** (la mayoría del OLAP).

> Regla de la semana: *si no lo ves en `EXPLAIN ANALYZE`, no lo creas.*

## Bibliografía

### DDIA (Kleppmann)
- **Cap. 3 — Storage and Retrieval**
  - OLTP vs OLAP: por qué en analítica duele leer “mucho”
  - Column-oriented thinking: por qué **leer menos columnas** y **ordenar por columnas consultadas** ayuda
  - Índices como tradeoff: aceleran ciertos accesos, cuestan en escritura/carga

### DuckDB
- Guía de Indexing (zonemaps + ART + selectividad y umbrales)
- `CREATE INDEX` statement
- `EXPLAIN ANALYZE` para verificar si un índice se usa

In [1]:
import os
os.chdir("..")
os.getcwd()

'c:\\Users\\hola-\\Documents\\FisicaComputacional3-Win\\de1-fundamentos-nasa-duckdb'

In [2]:
from pathlib import Path
import duckdb

PROJECT_ROOT = Path(".").resolve()
RAW_DIR = PROJECT_ROOT / "data" / "raw"
DB_PATH = PROJECT_ROOT / "data" / "exoplanets.duckdb"
ART_DIR = PROJECT_ROOT / "artifacts"
DOCS_DIR = PROJECT_ROOT / "docs"

RAW_DIR.mkdir(parents=True, exist_ok=True)
ART_DIR.mkdir(parents=True, exist_ok=True)
DOCS_DIR.mkdir(parents=True, exist_ok=True)

con = duckdb.connect(str(DB_PATH))

raw_csv = RAW_DIR / "pscomppars.csv"
if not raw_csv.exists():
    raise FileNotFoundError(f"No encuentro {raw_csv}. Necesitas el CSV de W01/W02.")

def sql_quote(s: str) -> str:
    return "'" + s.replace("'", "''") + "'"

# Bronze-lite
con.execute(f'''
CREATE OR REPLACE VIEW raw_ps AS
SELECT * FROM read_csv_auto({sql_quote(str(raw_csv.resolve()))})
''')

# Silver (mínimo) + Dim + Fact (idéntico W03B)
con.execute("DROP TABLE IF EXISTS silver_planet")
con.execute('''
CREATE TABLE silver_planet AS
SELECT
  pl_name,
  hostname,
  discoverymethod,
  disc_year,
  sy_snum,
  sy_pnum,
  sy_dist,
  ra,
  dec,
  pl_orbper,
  pl_rade,
  pl_bmasse,
  pl_eqt,
  st_teff,
  st_rad,
  st_mass
FROM raw_ps
WHERE pl_name IS NOT NULL
  AND hostname IS NOT NULL
  AND (disc_year IS NULL OR (disc_year BETWEEN 1980 AND 2026))
  AND (pl_rade  IS NULL OR (pl_rade  > 0 AND pl_rade  <= 30))
  AND (pl_bmasse IS NULL OR (pl_bmasse > 0))
''')

con.execute("DROP TABLE IF EXISTS dim_host_full")
con.execute('''
CREATE TABLE dim_host_full AS
SELECT
  hostname,
  MAX(sy_dist)  AS sy_dist,
  MAX(ra)       AS ra,
  MAX(dec)      AS dec,
  MAX(st_teff)  AS st_teff,
  MAX(st_rad)   AS st_rad,
  MAX(st_mass)  AS st_mass
FROM silver_planet
GROUP BY hostname
''')

con.execute("DROP TABLE IF EXISTS fact_planet")
con.execute('''
CREATE TABLE fact_planet AS
SELECT DISTINCT
  pl_name,
  hostname,
  discoverymethod,
  disc_year,
  pl_orbper,
  pl_rade,
  pl_bmasse,
  pl_eqt
FROM silver_planet
''')

con.sql("SELECT COUNT(*) AS n_fact, COUNT(DISTINCT pl_name) AS n_pl, COUNT(DISTINCT hostname) AS n_hosts FROM fact_planet").show()

┌────────┬───────┬─────────┐
│ n_fact │ n_pl  │ n_hosts │
│ int64  │ int64 │  int64  │
├────────┼───────┼─────────┤
│   6060 │  6060 │    4521 │
└────────┴───────┴─────────┘



## 0) Hacemos una tabla “más grande” (para que performance se note)

Vamos a multiplicar el tamaño repitiendo `fact_planet`.  
Esto NO es un truco “realista”, es solo para que el motor tenga algo que optimizar.

> Ajusta `MULTIPLIER` si tu máquina es lenta.

In [3]:
MULTIPLIER = 6000  # factor final ≈ (1 + MULTIPLIER) * n_fact

con.execute("DROP TABLE IF EXISTS fact_big_unsorted")
con.execute("CREATE TABLE fact_big_unsorted AS SELECT * FROM fact_planet")

for _ in range(MULTIPLIER):
    con.execute("INSERT INTO fact_big_unsorted SELECT * FROM fact_planet")

con.sql("SELECT COUNT(*) AS n_rows FROM fact_big_unsorted").show()

# Una copia ordenada por disc_year (para explotar mejor zonemaps)
con.execute("DROP TABLE IF EXISTS fact_big_sorted")
con.execute('''
CREATE TABLE fact_big_sorted AS
SELECT * FROM fact_big_unsorted
ORDER BY disc_year NULLS LAST
''')

con.sql("SELECT COUNT(*) AS n_rows FROM fact_big_sorted").show()

┌──────────┐
│  n_rows  │
│  int64   │
├──────────┤
│ 36366060 │
└──────────┘

┌──────────┐
│  n_rows  │
│  int64   │
├──────────┤
│ 36366060 │
└──────────┘



## 1) Zonemaps (min-max) + por qué el orden importa

DuckDB crea **zonemaps automáticamente** (min/max por bloques).  
Si tu filtro cae fuera del rango de un bloque, el motor puede **saltarse** bloques completos.

**Idea clave:** si los valores están “mezclados al azar”, los bloques tienen rangos amplios → no se puede saltar casi nada.  
Si ordenas por la columna filtrada, los rangos por bloque se vuelven estrechos → mejor “skip”.

Hoy no memorizamos teoría: lo comprobamos con `EXPLAIN ANALYZE`.

In [ ]:
# TU TURNO 1: cambia el rango de años y repite la comparación.
# Objetivo: ver si el plan/timing cambia entre unsorted y sorted.

q_unsorted = '''
-- TODO: tu consulta (fact_big_unsorted) con BETWEEN
SELECT COUNT(*) AS n
FROM fact_big_unsorted
WHERE disc_year IS NOT NULL
  AND disc_year BETWEEN 2010 AND 2012
'''

q_sorted = '''
-- TODO: la misma consulta pero sobre fact_big_sorted
SELECT COUNT(*) AS n
FROM fact_big_sorted
WHERE disc_year IS NOT NULL
  AND disc_year BETWEEN 2010 AND 2012
'''

print(con.sql("EXPLAIN ANALYZE " + q_unsorted).fetchone()[1])
print(con.sql("EXPLAIN ANALYZE " + q_sorted).fetchone()[1])


## 2) ART Index (`CREATE INDEX`) y selectividad

DuckDB también tiene índices tipo **ART** (Adaptive Radix Tree).  
Sirven especialmente para:
- igualdad (`=`) e `IN(...)`
- consultas **muy selectivas** (pocas filas)

**Importante:** en DuckDB, ART suele NO mejorar joins/aggregations/sorts.  
Por eso hay que verificar con `EXPLAIN ANALYZE` si se usa el index scan.

In [ ]:
# TU TURNO 2: crea un índice para una consulta MUY selectiva y comprueba el plan.
# Pistas:
# - usa igualdad: WHERE pl_name = '...'
# - primero mira el plan sin índice
# - luego CREATE INDEX ... ON fact_big_unsorted(pl_name)
# - mira el plan otra vez y busca 'INDEX' / 'Index Scan'

pl = con.sql("SELECT pl_name FROM fact_planet WHERE pl_name IS NOT NULL LIMIT 1").fetchone()[0]
pl_escaped = pl.replace("'", "''")

q_point = f'''
SELECT *
FROM fact_big_unsorted
WHERE pl_name = '{pl_escaped}'
'''

con.sql("EXPLAIN ANALYZE " + q_point).show()

# TODO: crea índice aquí (si no existe)
# con.execute("CREATE INDEX idx_fact_big_pl_name ON fact_big_unsorted(pl_name)")

con.sql("EXPLAIN ANALYZE " + q_point).show()

## 3) “¿Entonces creo índices para todo?” NO.

En DuckDB, los ART indexes:
- ayudan en **point/highly-selective queries**
- típicamente **no** cambian el costo de `JOIN + GROUP BY + ORDER BY`

Lo comprobamos: mismo query, con y sin índice, y miramos el plan.

In [ ]:
# TU TURNO 3: comprueba que un índice NO mejora (mucho) una agregación.
# - crea un índice en discoverymethod (opcional)
# - corre la agregación y mira el plan

q_agg = '''
SELECT discoverymethod, COUNT(*) AS n
FROM fact_big_unsorted
WHERE discoverymethod IS NOT NULL
GROUP BY discoverymethod
ORDER BY n DESC
LIMIT 10
'''

con.sql("EXPLAIN ANALYZE " + q_agg).show()

# TODO (opcional): crea índice en discoverymethod
# con.execute("CREATE INDEX idx_fact_big_disc ON fact_big_unsorted(discoverymethod)")

con.sql("EXPLAIN ANALYZE " + q_agg).show()

In [5]:
try:
    con.close()
    print("DuckDB connection closed.")
except NameError:
    print("No connection named 'con' in this notebook.")

DuckDB connection closed.


## Para entregar (W04B) — mesurado

### En clase
1) `docs/w04b_index_report.md` con:
   - (A) Rango `BETWEEN` en **unsorted** y **sorted** (pega ambos `EXPLAIN ANALYZE`)
   - (B) Punto `=` sobre `pl_name` antes/después de `CREATE INDEX` (pega ambos planes)
   - 2 conclusiones cortas: *cuándo ordenar ayuda* y *cuándo index ayuda*

2) `docs/decisions_log.md`: 1 decisión con evidencia:
   - “Creé/no creé índice en X porque…” + evidencia (plan o selectividad)

### Tarea
1) Una consulta tuya donde *intentes* usar índice (muy selectiva) y evidencia del plan.
2) Exporta a `artifacts/`:
   - `artifacts/w04b_explain_point.txt` (plan antes/después)

## Reflexión (bitácora)
- ¿Qué aprendiste sobre “selectividad”?
- En un warehouse real: ¿prefieres “ordenar/cluster” o “crear índices” y por qué?